In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
def unify_colname(col):
    return ".".join([el for el in col if el])


def evaluate(el):
    if not isinstance(el, str):
        return el
    if el[0] == "[" or el[0] == "'":
        return eval(el)
    return el


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = []
        for el in cell:
            if el == "nan" or pd.isna(el):
                nas.append(True)
            else:
                nas.append(False)
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_str(el: str):
    s = regex.sub(r"(^[\W]*)|([\W]*$)|([^\w:/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [5]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [6]:
# Counter(df_in.columns.tolist()+df_ou.columns.tolist()+df_mi.columns.tolist())

In [7]:
# df[
#     "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode"
# ]

In [8]:
# print(sorted(list(set(list(df_in.columns) + list(df_ou.columns) + list(df_mi.columns))), key=len))

In [9]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [10]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

In [11]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

In [12]:
# tend_cat2 = pd.read_csv(r"C:\Users\josea\Downloads\empresas2.csv")
tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_24564\886581414.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [13]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

In [14]:
tender_filt = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
tender_filt["id"] = tender_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# tender_filt["winner"] = tender_filt["winner"].apply(evaluate)
tender_filt["cpv"] = tender_filt["cpv"].apply(evaluate)
tender_filt = tender_filt.explode("cpv")  # .explode("winner")
tender_filt["cpv"] = tender_filt["cpv"].astype(str).apply(process_cpv)
tender_filt["location"] = tender_filt["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
tender_filt = (
    tender_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tender_filt["cpv_div"] = tender_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)

In [15]:
agg_tender = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .groupby(["id"])
    .agg(list)
)
valid_agg_tender = agg_tender[agg_tender["title"].apply(lambda x: len(set(x)) == 1)]
valid_agg_tender = valid_agg_tender.applymap(
    lambda x: Counter(x).most_common()[0][0]
).reset_index()
print(valid_agg_tender.shape)
display(valid_agg_tender.head())

(245975, 6)


,id,title,summary,cpv,location,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,id licitación: 0/2020-5/2020 órgano de contrat...,454541000,nan,45
1,00-0000,licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,791000000,nan,79
2,00-2019,comunicació de contractació anual programada e...,id licitación: 00/2019 órgano de contratación:...,853200000,nan,85
3,00-d-12-0000572,servicio de mantenimiento del sistema de alime...,id licitación: 00/d/12/0000572 órgano de contr...,505320000,nan,50
4,00-d-12-421,servicio de mantenimiento de la licencia dynat...,id licitación: 00/d/12/421 órgano de contratac...,48700000,nan,48


In [16]:
# tender_filt["id_clean"] = tender_filt["id"].apply(
#     lambda x: unidecode.unidecode(regex.sub(r"\W", "", x)) if not pd.isna(x) else np.nan
# )
# agg_tender2 = (
#     tender_filt[
#         tender_filt["location"].str.contains(
#             "|".join(
#                 [
#                     "nan",
#                     "espana",
#                     "cataluna",
#                     "catalunya",
#                     "barcelona",
#                     "tarragona",
#                     "girona",
#                     "gerona",
#                     "lleida",
#                     "lerida",
#                 ]
#             )
#         )
#     ]
#     .groupby(["id_clean"])
#     .agg(list)
# )
# valid_agg_tender2 = agg_tender2[agg_tender2["title"].apply(lambda x: len(set(x)) == 1)]
# valid_agg_tender2 = valid_agg_tender2.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
# print(valid_agg_tender.shape)
# display(valid_agg_tender.head())

In [17]:
# print(valid_agg_tender2.shape)
# display(valid_agg_tender2.head())

In [18]:
# gencat_ids = (
#     gencat[
#         [
#             "title",
#             "ContractFolderStatus.ContractFolderID",
#             "ContractFolderStatus.ProcurementProject.Name",
#             "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
#             "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
#         ]
#     ]
#     .rename(columns=rename_columns)
#     .applymap(str.lower)
# )

In [19]:
# Clean tenders cat
tend_cat_filt = (
    tend_cat[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
    .applymap(process_str)
)
tend_cat_filt["id"] = tend_cat_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
tend_cat_filt["cpv"] = tend_cat_filt["cpv"].astype(str).apply(process_cpv)
tend_cat_filt = (
    tend_cat_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tend_cat_filt["cpv_div"] = tend_cat_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)

In [20]:
tend_cat_filt.head()

,title,summary,id,cpv,cpv_div
0,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
1,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
2,acord marc per al subministrament i installaci...,subministrament i installació de mobiliari de ...,ccs-2022-6,391000003,39
3,publicació agregada contractes menors dacc 1r ...,compra de dues minicabines de seguretat biològica,ag-2023-662,380000005,38
4,publicació agregada contractes menors dacc 1r ...,redacció pla dautoprotecció de lembassament de...,ag-2023-663,713560008,71


In [63]:
# tend_cat_filt[tend_cat_filt["id"]=="ccs-2022-6"]
# c = []
# for el in tend_cat[:3].apply(lambda x: Counter(x), axis=0):
#     c.append(3 not in el.values())
# tend_cat[:3][tend_cat.columns[c]]

In [22]:
# Aggregate by ID
valid_tend_cat = tend_cat_filt.groupby("id").agg(list)
valid_tend_cat = valid_tend_cat[
    valid_tend_cat["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat = valid_tend_cat.applymap(
    lambda x: Counter(x).most_common()[0][0]
).reset_index()
print(len(valid_tend_cat))
display(valid_tend_cat.head())

227651


,id,title,summary,cpv,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,45
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79
2,00-0000,licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,79
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN
4,000-expedient-2020-034,000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,92


In [36]:
tend_cat[tend_cat["denominacio"].str.lower() == "rehabilitació de casa de poblet"][
    tend_cat.columns[10:]
]

,codi_expedient,tipus_contracte,procediment,fase_publicacio,denominacio,objecte_contracte,valor_estimat_contracte,codi_nuts,lloc_execucio,durada_contracte,...,tipus_empresa,url_json_futura,url_json_cpm,url_json_previ,url_json_licitacio,url_json_avaluacio,url_json_adjudicacio,url_json_formalitzacio,url_json_anulacio,url_json_agregada
153086,0/2020-5/2020,Obres,Obert Simplificat,Adjudicació,Rehabilitació de Casa de Poblet,Rehabilitació de Casa de Poblet,167714.63,ES513,Lleida,15/12/2021 a 15/04/2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
valid_tend_cat

,id,title,summary,cpv,cpv_div
0,0-2020-5-2020,rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,45
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79
2,00-0000,licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,79
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN
4,000-expedient-2020-034,000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,92
...,...,...,...,...,...
227646,z8489,subministrament i installació de lescomesa de ...,subministrament i installació de lescomesa de ...,515000007,51
227647,z8655,obres dinstallació duna planta fotovoltaica pe...,obres dinstallació duna planta fotovoltaica pe...,317123319,31
227648,z8672,obres dampliació de la xarxa del tub verd per ...,obres dampliació de la xarxa del tub verd per ...,452200005,45
227649,zls23-0084,2023 1r trimestre contractes menors associació...,traduccions,795300008,79


In [ ]:
tend_cat_filt.groupby("id").agg(list)

# TODO
- Hacer match con budget

In [37]:
# Match tenders by ID
matched_id = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=["id"],
    right_on=["id"],
)
matched_id = matched_id[
    [
        "id",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
    ]
]
title = []
cpv_div = []
for tx, ty, cx, cy in matched_id[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    title.append(Counter(set([tx, ty])))
    cpv_div.append(Counter(set([cx, cy])))

matched_id["title"] = title
matched_id["cpv_div"] = cpv_div

matched_id = matched_id[["id", "title", "cpv_div"]]
print(len(matched_id))
display(matched_id.head())

83775


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,"{'licitació de prova sobre digital': 1, 'licit...",{'79': 1}
2,000-expedient-2020-034,{'000 expedient 2020 034 - licitació de la con...,{'92': 1}
3,000-expedient-2021-266,{'el present expedient té per objecte el submi...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [161]:
# Match tenders by title and CPV
matched_aux = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
)
matched_aux = matched_aux[
    [
        "id_x",
        "id_y",
        "title",
        "cpv_div",
    ]
]
matched_aux = matched_aux[matched_aux["id_x"] != matched_aux["id_y"]]

# # ERROR
# matched_aux[
#     (~matched_aux["id_x"].isin(matched_id["id"]))
#     & (~matched_aux["id_y"].isin(matched_id["id"]))
# ]


# m_x = matched_aux[~matched_aux["id_x"].isin(matched_aux["id_y"])].rename({"id_x":"id"}, axis=1)
# m_y = matched_aux[~matched_aux["id_y"].isin(matched_aux["id_x"])].rename({"id_y":"id"}, axis=1)
# matched_aux = pd.merge(
#     m_x,
#     m_y,
#     how="outer",
#     left_on="id",
#     right_on="id",
# )

# title = []
# cpv_div = []
# for tx, ty, cx, cy in matched_aux[["title_x", "title_y", "cpv_div_x", "cpv_div_y"]].values:
#     title.append(Counter(set([tx, ty])-set([np.nan])))
#     cpv_div.append(Counter(set([cx, cy])-set([np.nan])))
#     # t = dict()
#     # c = dict()
#     # t.update(tx if not pd.isna(tx) else dict())
#     # t.update(ty if not pd.isna(ty) else dict())
#     # c.update(cx if not pd.isna(cx) else dict())
#     # c.update(cy if not pd.isna(cy) else dict())
#     # title.append(t)
#     # cpv_div.append(c)

# matched_aux["title"] = title
# matched_aux["cpv_div"] = cpv_div

# matched_aux = matched_aux[["id", "title", "cpv_div"]]
# print(len(matched_aux))
# display(matched_aux.head())

In [155]:
matched_aux

,id_x,id_y,title,cpv_div
19,001-21000133,001-21002534,servei dassessorament mediació i administració...,66
20,001-21002534,001-21000133,servei dassessorament mediació i administració...,66
24,001-21000518,001-22002143,realització de les adaptacions de la creativit...,79
25,001-22002143,001-21000518,realització de les adaptacions de la creativit...,79
46,21000793,001-21002039,servei de suport i assistència tècnica pel con...,71
...,...,...,...,...
50356,x2021000949,x2021002529,servei de poda darbrat corresponent als exempl...,77
50415,x2022001598,x2020001177,contractació del servei de manteniment de la p...,50
50417,x2022001599,x2021001963,contractació del servei de socorrisme de la pi...,92
50448,x939-2020,ge-6172-2017,prestació del servei de neteja dels edificis m...,90


In [93]:
# sum(matched_aux["title"].apply(lambda x: sum(x.values())))

In [165]:
m_x

,id,title,cpv_div
109,003-23,contractació dun servei que inclogui 3 persone...,72
110,003-23,contractació dun servei que inclogui 3 persone...,72
122,01-04-07-14-18,disseny muntatge i execució de lactivitat del ...,79
123,01-04-07-17-2018,contractació del servei dactualització tecnolò...,48
124,01-04-07-18-2018,programa dacollida per a nous ciutadans,98
...,...,...,...
49924,tnc-2019-00041,contractació de la directora descena per lobra...,92
49925,tnc-2019-00042,contractació de lescenògraf enric planas terue...,92
49967,tnc-2020-00001,contractació de la conquesta del pol sud scp p...,92
50199,x2023000113,servei de recollida danimals de companyia aban...,98


In [167]:
m_x = matched_aux.loc[
    ~matched_aux["id_x"].isin(matched_id["id"]), ["id_x", "title", "cpv_div"]
].rename({"id_x": "id"}, axis=1)
m_y = matched_aux.loc[
    ~matched_aux["id_y"].isin(matched_id["id"]), ["id_y", "title", "cpv_div"]
].rename({"id_y": "id"}, axis=1)
a = pd.concat([m_x, m_y]).groupby("id").agg(Counter).reset_index()
a

,id,title,cpv_div
0,001-22,{'contractació dun servei que inclogui 3 perso...,{'72': 1}
1,002-20,{'contractació dun servei que inclogui 3 perso...,{'72': 1}
2,003-23,{'contractació dun servei que inclogui 3 perso...,{'72': 2}
3,0047,{'servei de manteniment preventiu i correctiu ...,{'50': 1}
4,006-19,{'maquetació de la revista informat versió en ...,{'79': 1}
...,...,...,...
2968,uvic-ucc-201704,{'servei postal dels diversos centres de la un...,{'60': 1}
2969,x2021000479,{'subministrament de material informàtic': 2},{'30': 2}
2970,x2023000113,{'servei de recollida danimals de companyia ab...,{'98': 2}
2971,x2023000547,{'contracte de serveis de neteja dels edificis...,{'90': 1}


In [170]:
a["title"]

0       {'contractació dun servei que inclogui 3 perso...
1       {'contractació dun servei que inclogui 3 perso...
2       {'contractació dun servei que inclogui 3 perso...
3       {'servei de manteniment preventiu i correctiu ...
4       {'maquetació de la revista informat versió en ...
                              ...                        
2968    {'servei postal dels diversos centres de la un...
2969        {'subministrament de material informàtic': 2}
2970    {'servei de recollida danimals de companyia ab...
2971    {'contracte de serveis de neteja dels edificis...
2972    {'servei de neteja de les dependències municip...
Name: title, Length: 2973, dtype: object

In [120]:
# # m_x = matched_aux[["id_x", "title", "cpv_div"]].rename({"id_x":"id"}, axis=1)
# # m_y = matched_aux[["id_y", "title", "cpv_div"]].rename({"id_y":"id"}, axis=1)
m_x = matched_aux[
    ~(
        matched_aux["id_x"].isin(matched_aux["id_y"])
        | matched_aux["id_x"].isin(matched_id["id"])
    )
].rename({"id_x": "id"}, axis=1)
m_y = matched_aux[
    ~(
        matched_aux["id_y"].isin(matched_aux["id_x"])
        | matched_aux["id_y"].isin(matched_id["id"])
    )
].rename({"id_y": "id"}, axis=1)
a = pd.merge(
    m_x,
    m_y,
    how="outer",
    left_on="id",
    right_on="id",
)

In [122]:
m_x

,id,title,summary_x,cpv_x,location,cpv_div,id_y,summary_y,cpv_y
109,003-23,contractació dun servei que inclogui 3 persone...,id licitación: 003/23 órgano de contratación: ...,726100000,nan,72,001-22,contractació dun servei que inclogui 3 persone...,726100009
110,003-23,contractació dun servei que inclogui 3 persone...,id licitación: 003/23 órgano de contratación: ...,726100000,nan,72,002-20,contractació dun servei que inclogui 3 persone...,726100009
122,01-04-07-14-18,disseny muntatge i execució de lactivitat del ...,id licitación: 010407-14/18 órgano de contrata...,799521000,nan,79,010407-14-18,disseny muntatge i execució de lactivitat del ...,799521003
123,01-04-07-17-2018,contractació del servei dactualització tecnolò...,id licitación: 010407-17-2018 órgano de contra...,480000000,nan,48,010407-17-2018,contractació del servei dactualització tecnolò...,480000008
124,01-04-07-18-2018,programa dacollida per a nous ciutadans,id licitación: 010407-18/2018 órgano de contra...,980000000,nan,98,010407-18-2018,programa dacollida per a nous ciutadans,980000003
...,...,...,...,...,...,...,...,...,...
49924,tnc-2019-00041,contractació de la directora descena per lobra...,id licitación: tnc201900041 órgano de contrata...,923120000,nan,92,tnc201900041,contractació de la directora descena per lobra...,923120001
49925,tnc-2019-00042,contractació de lescenògraf enric planas terue...,id licitación: tnc201900042 órgano de contrata...,923120000,nan,92,tnc201900042,contractació de lescenògraf enric planas terue...,923120001
49967,tnc-2020-00001,contractació de la conquesta del pol sud scp p...,id licitación: tnc202000001 órgano de contrata...,923120000,nan,92,tnc202000001,contractació de la conquesta del pol sud scp p...,923120001
50199,x2023000113,servei de recollida danimals de companyia aban...,id licitación: x2023000113 órgano de contratac...,983800000,nan,98,x2019002242,servei de recollida danimals de companyia aban...,983800000


In [121]:
a

,id,title_x,summary_x_x,cpv_x_x,location_x,cpv_div_x,id_y,summary_y_x,cpv_y_x,id_x,title_y,summary_x_y,cpv_x_y,location_y,cpv_div_y,summary_y_y,cpv_y_y
0,003-23,contractació dun servei que inclogui 3 persone...,id licitación: 003/23 órgano de contratación: ...,726100000,nan,72,001-22,contractació dun servei que inclogui 3 persone...,726100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,003-23,contractació dun servei que inclogui 3 persone...,id licitación: 003/23 órgano de contratación: ...,726100000,nan,72,002-20,contractació dun servei que inclogui 3 persone...,726100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-04-07-14-18,disseny muntatge i execució de lactivitat del ...,id licitación: 010407-14/18 órgano de contrata...,799521000,nan,79,010407-14-18,disseny muntatge i execució de lactivitat del ...,799521003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-04-07-17-2018,contractació del servei dactualització tecnolò...,id licitación: 010407-17-2018 órgano de contra...,480000000,nan,48,010407-17-2018,contractació del servei dactualització tecnolò...,480000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-04-07-18-2018,programa dacollida per a nous ciutadans,id licitación: 010407-18/2018 órgano de contra...,980000000,nan,98,010407-18-2018,programa dacollida per a nous ciutadans,980000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,tnc201900041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tnc-2019-00041,contractació de la directora descena per lobra...,id licitación: tnc201900041 órgano de contrata...,923120000,nan,92,contractació de la directora descena per lobra...,923120001
9832,tnc201900042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tnc-2019-00042,contractació de lescenògraf enric planas terue...,id licitación: tnc201900042 órgano de contrata...,923120000,nan,92,contractació de lescenògraf enric planas terue...,923120001
9833,tnc202000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tnc-2020-00001,contractació de la conquesta del pol sud scp p...,id licitación: tnc202000001 órgano de contrata...,923120000,nan,92,contractació de la conquesta del pol sud scp p...,923120001
9834,uvic-ucc-201704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uvic-ucc-201801,servei postal dels diversos centres de la univ...,id licitación: uvic-ucc-201801 órgano de contr...,604110000,nan,60,servei postal dels diversos centres de la univ...,604110002


In [115]:
matched_id[matched_id["id"] == "21000793"]

,id,title,cpv_div
27022,21000793,{'servei de suport i assistència tècnica pel c...,"{nan: 1, '71': 1}"


In [113]:
a[a["id"] == "001-21002534"]

,id,id_y,title_x,cpv_div_x,id_x,title_y,cpv_div_y


In [60]:
m_x[m_x["id_y"] == "x939-2020"]

,id_x,id_y,title,cpv_div
49457,scpo2023000001,x939-2020,contracte del servei de neteja dedificis munic...,90


In [54]:
m_y

,id_x,id_y,title,cpv_div
46,21000793,001-21002039,servei de suport i assistència tècnica pel con...,71
58,001-21003081,18002596,subministrament de material de conservació i p...,30
81,001-22002887,20002832-20010021,assistència tècnica per al seguiment i coordin...,71
85,001-22003093,001-21001877,gestió de la casa de laigua de trinitat nova a...,98
94,001-23,006-19,maquetació de la revista informat versió en ca...,79
...,...,...,...,...
50356,x2021000949,x2021002529,servei de poda darbrat corresponent als exempl...,77
50415,x2022001598,x2020001177,contractació del servei de manteniment de la p...,50
50417,x2022001599,x2021001963,contractació del servei de socorrisme de la pi...,92
50448,x939-2020,ge-6172-2017,prestació del servei de neteja dels edificis m...,90


In [109]:
matched_aux

,id,title_x,cpv_div_x,title_y,cpv_div_y
0,001-21000133,servei dassessorament mediació i administració...,66,servei dassessorament mediació i administració...,66
1,001-21002534,servei dassessorament mediació i administració...,66,servei dassessorament mediació i administració...,66
2,001-21000518,realització de les adaptacions de la creativit...,79,realització de les adaptacions de la creativit...,79
3,001-22002143,realització de les adaptacions de la creativit...,79,realització de les adaptacions de la creativit...,79
4,21000793,servei de suport i assistència tècnica pel con...,71,NaN,NaN
...,...,...,...,...,...
1600376,x2021002529,NaN,NaN,servei de poda darbrat corresponent als exempl...,77
1600377,x2020001177,NaN,NaN,contractació del servei de manteniment de la p...,50
1600378,x2021001963,NaN,NaN,contractació del servei de socorrisme de la pi...,92
1600379,ge-6172-2017,NaN,NaN,prestació del servei de neteja dels edificis m...,90


In [98]:
# Combine both
matched_final = pd.merge(
    matched_id,
    matched_aux,
    how="outer",
    left_on="id",
    right_on="id",
)
title = []
cpv_div = []
for tx, ty, cx, cy in matched_final[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    t = dict()
    c = dict()
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    title.append(t)
    cpv_div.append(c)

matched_final["title"] = title
matched_final["cpv_div"] = cpv_div

matched_final = matched_final[["id", "title", "cpv_div"]]
print(len(matched_final))
display(matched_final.head())

1680171


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,"{'licitació de prova sobre digital': 1, 'licit...",{'79': 1}
2,000-expedient-2020-034,{'000 expedient 2020 034 - licitació de la con...,{'92': 1}
3,000-expedient-2021-266,{'el present expedient té per objecte el submi...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [68]:
# matched_final[matched_final["title"].apply(lambda x: sum(x.values())>1)].values

In [99]:
# Unique
print(len(matched_final) / len(valid_tend_cat))
# Total
print(
    matched_final["title"].apply(lambda x: sum(x.values())).sum() / len(valid_tend_cat)
)

7.380468348480789
7.713552762781626
